# Dectecting Heart Disease Machine Learning Project

#### Citation:
This project uses data from the UCI Machine Learning Repository


Janosi,Andras, Steinbrunn,William, Pfisterer,Matthias, and Detrano,Robert. (1988). Heart Disease. UCI Machine Learning Repository. https://doi.org/10.24432/C52P4X.

### Imports

In [ ]:
import pandas as pd

In [4]:
feat_names = ['age', # (numerical)
              'sex', # (binary)
              'cp', # (categorical) [1-4] chest pain 
              'trestbps', # (numerical) resting blood pressure 
              'chol', # (numerical) serum cholestoral in mg/dl 
              'fbs', # (binary) fasting blood sugar > 120 mg/dl 
              'restecg', # (ordinal) [0-2] resting electrocardiographic results 
              'thalach', # (numerical) maximum heart rate acheived 
              'exang', # (binary) exercised induced angina 
              'oldpeak', # (numerical) ST depression induced by exercise relative to rest 
              'slope', # (ordinal) [1-3] slope of the peak exercise ST segment
              'ca', # (numerical) number of major vessels colored by flourosopy
              'thal', # (ordinal?) "3 = normal; 6 = fixed defect; 7 = reversable defect"
              'num'] # (categorical) [0-4] diagnosis 
heart_data = pd.read_csv('cleveland_heart_data', names = feat_names)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,45.0,1.0,1.0,110.0,264.0,0.0,0.0,132.0,0.0,1.2,2.0,0.0,7.0,1
299,68.0,1.0,4.0,144.0,193.0,1.0,0.0,141.0,0.0,3.4,2.0,2.0,7.0,2
300,57.0,1.0,4.0,130.0,131.0,0.0,0.0,115.0,1.0,1.2,2.0,1.0,7.0,3
301,57.0,0.0,2.0,130.0,236.0,0.0,2.0,174.0,0.0,0.0,2.0,1.0,3.0,1


## Data Cleaning and Feature Engineering